In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

desired_width = 600
pd.set_option('display.width', desired_width)

input_file = ['../raw_data/virginia.xls', '../raw_data/maryland.xls']

crime_va, crime_maryland = pd.read_excel(input_file[0]), pd.read_excel(input_file[1])

############### drop description rows and columns
crime_va.drop([0, 1, 2, 3, 4, 100, 101], axis=0, inplace=True)
crime_va.drop(['Table 8', 'Unnamed: 5'], axis=1, inplace=True)
crime_maryland.drop([0,1,2,3,4,27, 28,29], axis=0, inplace=True)
crime_maryland.drop(['MARYLAND', 'Unnamed: 5'], axis=1, inplace=True)

############# set column names
columns = ['County', 'Violent', 'Murder_and_nonnegligent_manslaughter',
           'Rape','Robbery', 'Aggravated_assault', 'Property_crime',
           'Burglary', 'Larceny_theft', 'Motor_vehicle_theft', 'Arson', 'Population']
crime_va.columns = columns
crime_maryland.columns = columns

crime_DF = pd.concat([crime_va, crime_maryland], ignore_index=True)

print('columns:', crime_DF.columns)
print('len:', len(crime_DF))
crime_DF.head()

columns: Index(['County', 'Violent', 'Murder_and_nonnegligent_manslaughter', 'Rape', 'Robbery', 'Aggravated_assault', 'Property_crime', 'Burglary', 'Larceny_theft', 'Motor_vehicle_theft', 'Arson', 'Population'], dtype='object')
len: 117


,County,Violent,Murder_and_nonnegligent_manslaughter,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson,Population
0,Albemarle County Police Department,88,1,29,21,37,1271,145,1061,65,8,105715
1,Amelia,18,2,6,2,8,125,28,87,10,0,12856
2,Amherst,32,1,13,6,12,279,35,222,22,2,29930
3,Appomattox,14,0,5,2,7,129,18,107,4,0,15388
4,Arlington County Police Department,363,1,56,103,203,3252,170,2912,170,5,236691


In [4]:
########## clean county name and set index
crime_DF.sort_values(by='County', inplace=True)
## remove if exist
s = ' County Police Department'
for n, c in enumerate(crime_DF.County.values):
    if c.endswith(s):
        crime_DF.County.values[n] = c[:-len(s)]
# crime_DF.index = np.arange(len(crime_DF))
crime_DF.index = crime_DF.County.values
crime_DF.drop(['County'], axis=1, inplace=True)

### change data store type
crime_DF = crime_DF.astype(np.float)

### calculate crime rate (per 100,000 people)
for c in crime_DF.columns[0:-1]:
    crime_DF.loc[:, c] = crime_DF.loc[:, c] / crime_DF.iloc[:, -1] * 100000

crime_DF.describe()

,Violent,Murder_and_nonnegligent_manslaughter,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson,Population
count,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,1.170000e+02
mean,126.506339,3.389329,28.794481,16.195324,78.127205,993.898559,195.127665,742.085875,56.685019,6.799976,9.189762e+04
std,72.921599,5.851991,23.609888,23.677726,51.506226,473.172214,94.854378,400.189359,36.119282,7.553346,1.905791e+05
min,1.101262,0.000000,0.000000,0.000000,0.000000,0.110126,0.000000,0.110126,0.000000,0.000000,2.300000e+03
25%,81.984013,0.000000,12.140720,0.000000,45.383817,691.860845,122.450541,476.592445,33.848584,0.000000,1.476400e+04
50%,111.195350,1.166956,24.393644,8.119519,63.298770,919.896641,182.255595,638.562053,55.897149,4.647056,2.635900e+04
75%,158.829830,3.935149,40.184229,20.671835,103.012570,1200.729855,261.569416,896.161493,71.823601,10.108668,7.090400e+04
max,529.104986,33.866161,133.795057,180.138768,306.488606,2809.057719,517.649756,2243.380723,248.367680,33.866161,1.096200e+06


In [17]:
crime_DF.head()

,Violent,Murder_and_nonnegligent_manslaughter,Rape,Robbery,Aggravated_assault,Property_crime,Burglary,Larceny_theft,Motor_vehicle_theft,Arson,Population
Accomack,158.829830,18.468585,59.099472,18.468585,62.793189,1399.918738,413.696303,890.185794,96.036642,3.693717,27073.0
Albemarle,83.242681,0.945940,27.432247,19.864731,34.999764,1202.289174,137.161235,1003.641867,61.486071,7.567516,105715.0
Alleghany,101.505667,0.000000,25.376417,8.458806,67.670445,583.657588,236.846557,287.599391,59.211639,8.458806,11822.0
Amelia,140.012446,15.556938,46.670815,15.556938,62.227754,972.308650,217.797138,676.726820,77.784692,0.000000,12856.0
Amherst,106.916138,3.341129,43.434681,20.046776,40.093552,932.175075,116.939526,741.730705,73.504845,6.682259,29930.0


In [9]:
# violin plot of population
fig = px.violin(crime_DF, y="Population", box=True, points='all')
fig.update_layout(
    title='County Population Distribution',
    yaxis_title='Population'
)
fig.show()

In [7]:
# check distribution of different violent crime types
fig = go.Figure()

fig.add_trace(go.Box(y=crime_DF.Rape, name='Rape', boxpoints='all', jitter=0.3, pointpos=-1.8))
fig.add_trace(go.Box(y=crime_DF.Robbery, name='Robbery', boxpoints='all', jitter=0.3, pointpos=-1.8))
fig.add_trace(go.Box(y=crime_DF.Murder_and_nonnegligent_manslaughter,
                     name='Murder_and_nonnegligent_manslaughter', boxpoints='all', jitter=0.3, pointpos=-1.8))
fig.add_trace(go.Box(y=crime_DF.Aggravated_assault, 
                     name='Aggravated_assault', boxpoints='all', jitter=0.3, pointpos=-1.8))

fig.update_layout(
    title='Violent crime distribution',
    yaxis_title='crime per 100,000 inhabitant',
    
)
fig.show()

In [8]:
# check distribution of different property crime types
fig = go.Figure()
# fig.add_trace(go.Box(y=crime_DF.Burglary, x=crime_DF.Pop_Gp, name='Burglary',
#                      boxpoints='all', jitter=0.3, pointpos=-1.8))
# fig.add_trace(go.Box(y=crime_DF.Larceny_theft, x=crime_DF.Pop_Gp, name='Larceny_theft',
#                      boxpoints='all', jitter=0.3, pointpos=-1.8))
# fig.add_trace(go.Box(y=crime_DF.Motor_vehicle_theft, x=crime_DF.Pop_Gp, name='Motor_vehicle_theft',
#                      boxpoints='all', jitter=0.3, pointpos=-1.8))

fig.add_trace(go.Box(y=crime_DF.Burglary, name='Burglary',
                     boxpoints='all', jitter=0.3, pointpos=-1.8))
fig.add_trace(go.Box(y=crime_DF.Larceny_theft, name='Larceny_theft',
                     boxpoints='all', jitter=0.3, pointpos=-1.8))
fig.add_trace(go.Box(y=crime_DF.Motor_vehicle_theft, name='Motor_vehicle_theft',
                     boxpoints='all', jitter=0.3, pointpos=-1.8))


fig.update_layout(
    title='Property crime distribution',
    yaxis_title='crime per 100,000 inhabitant',
    
)
fig.show()

In [63]:
# bar chart for 25% counties with lowest violent rate and 
# 25% counties with highest violent rate
num_total = len(crime_DF)
p_25 = np.int(0.25*num_total)
p_75 = np.int(0.75*num_total)
sorted_by_violent = crime_DF.sort_values(by='Violent')
violent_low = sorted_by_violent.iloc[0:p_25]
violent_high = sorted_by_violent.iloc[p_75:]

types = ['Robbery', 'Rape', 'Murder', 'Assault']
means_lo = violent_low.mean()
means_hi = violent_high.mean()
violent_means_lo = means_lo.loc[['Robbery','Rape',
                                      'Murder_and_nonnegligent_manslaughter',
                                      'Aggravated_assault']].values
violent_means_hi = means_hi.loc[['Robbery','Rape',
                                      'Murder_and_nonnegligent_manslaughter',
                                      'Aggravated_assault']].values

fig = go.Figure(data=[
    go.Bar(name='Low-violent-rate Counties', x=types, y=violent_means_lo),
    go.Bar(name='High-violent-rate Counties', x=types, y=violent_means_hi)
])
# Change the bar mode
fig.update_layout(title='Violent Crime Comparison',barmode='group')
fig.show()

ratio = violent_means_hi/violent_means_lo
fig = go.Figure(data=[
    go.Bar(name='Ratio', x=types, y=ratio, text=["{r:.2f}".format(r=r) for r in ratio],
           textposition='auto')
])
# Change the bar mode
fig.update_layout(title='Violent Crime rate ratio')
fig.show()

# fig = go.Figure()
# fig.add_trace(go.Bar(
#     x=sorted_by_violent.index,
#     y=sorted_by_violent.Robbery,
#     name='Robbery',
# ))
# fig.add_trace(go.Bar(
#     x=sorted_by_violent.index,
#     y=sorted_by_violent.Rape,
#     name='Rape'
# ))
# fig.add_trace(go.Bar(
#     x=sorted_by_violent.index,
#     y=sorted_by_violent.Murder_and_nonnegligent_manslaughter,
#     name='Murder_and_nonnegligent_manslaughter'
# ))
# fig.add_trace(go.Bar(
#     x=sorted_by_violent.index,
#     y=sorted_by_violent.Aggravated_assault,
#     name='Aggravated_assault'
# ))
# fig.show()

In [64]:
sorted_by_property = crime_DF.sort_values(by='Property_crime')
property_lo = sorted_by_property.iloc[0:p_25]
property_hi = sorted_by_property.iloc[p_75:]

types = ['Burglary', 'Larceny_theft', 'Motor_vehicle_theft']
pmeans_lo = property_lo.mean()
pmeans_hi = property_hi.mean()
property_means_lo = pmeans_lo.loc[['Burglary','Larceny_theft','Motor_vehicle_theft']].values
property_means_hi = pmeans_hi.loc[['Burglary','Larceny_theft','Motor_vehicle_theft']].values

fig = go.Figure(data=[
    go.Bar(name='Low-property-rate Counties', x=types, y=property_means_lo),
    go.Bar(name='High-property-rate Counties', x=types, y=property_means_hi)
])
# Change the bar mode
fig.update_layout(title='Property Crime Comparison',barmode='group')
fig.show()

ratio = property_means_hi/property_means_lo
fig = go.Figure(data=[
    go.Bar(name='Ratio', x=types, y=ratio, text=["{r:.2f}".format(r=r) for r in ratio],
           textposition='auto')
])
# Change the bar mode
fig.update_layout(title='Property Crime rate ratio')
fig.show()

In [62]:
["{r:.2f}".format(r=r) for r in ratio]

['7.44', '3.15', '3.40', '4.01']

In [34]:
violent_high.mean()

Violent                                    217.380132
Murder_and_nonnegligent_manslaughter         5.686019
Rape                                        41.474562
Robbery                                     33.779040
Aggravated_assault                         136.440511
Property_crime                            1394.415662
Burglary                                   251.905765
Larceny_theft                             1062.788909
Motor_vehicle_theft                         79.720989
Arson                                        8.758438
Population                              152993.433333
dtype: float64